In [1]:
# !pip install ds11mltoolkit
# import ds11mltoolkit

In [2]:
import numpy as np
import pandas as pd
import datetime
import pickle
import os

from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
root_path = '.'

## Google Colab

In [4]:
from google.colab import drive
drive.mount('/content/drive')

root_path = '/content/drive/MyDrive/projects/facial_feeling_detection'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##### Check GPU

In [5]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5999836527229922724
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14391508992
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11117991836438541972
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

## Constants

In [4]:
SEED = 42
LABELS = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}
TRAIN_PATH = root_path + '/data/train/'
TEST_PATH = root_path + '/data/test/'
MODELS_PATH = root_path + '/models/'
SUBMISSIONS_PATH = root_path + '/submissions/'

## Data Loading

In [5]:
data = np.load(root_path + '/data/data.npz')
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']

with open(root_path + '/data/test_ids.pkl', 'rb') as f:
    test_ids = pickle.load(f)

submission_ids = pd.read_csv(root_path + '/data/test_set.csv')

## Model

In [6]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dropout(.2),
    Dense(512, activation='relu'),
    Dense(7, activation='softmax')
])

In [7]:
model.compile(
    optimizer='SGD',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# print(model.summary())

In [8]:
callback_path = MODELS_PATH + 'callback.h5'

earlystop = EarlyStopping(patience=5)
mcheckpoint = ModelCheckpoint(
    callback_path,
    monitor='val_accuracy',
    save_best_only=True
)

In [9]:
history = model.fit(
    X_train, y_train,
    validation_split=.2,
    epochs=1000,
    batch_size=32,
    # callbacks=[earlystop, mcheckpoint]
    callbacks=[mcheckpoint]
)

Epoch 1/2
1403/1403 [==============================] - 77s 54ms/step - loss: 1.8870 - accuracy: 0.2664 - val_loss: 1.7951 - val_accuracy: 0.2877
Epoch 2/2
1403/1403 [==============================] - 77s 55ms/step - loss: 1.7842 - accuracy: 0.2892 - val_loss: 1.7829 - val_accuracy: 0.2803


In [10]:
max_val_accuracy = max(history.history['val_accuracy'])
date = datetime.datetime.now().strftime('%y%m%d%H%M%S')
model_path = f'{MODELS_PATH}model_val_acc_{max_val_accuracy:.4f}_{date}.h5'

os.rename(callback_path, model_path)

## Prediction/Submission

In [11]:
# model_path = MODELS_PATH + 'model_val_acc_0.4901_230224172910.h5'

model_last = load_model(model_path)

In [12]:
y_pred = model_last.predict(X_test)
predictions = [np.argmax(pred) for pred in y_pred]

221/221 [==============================] - 3s 13ms/step


In [13]:
submission_pred = pd.DataFrame(test_ids, columns=['id_img']).astype(int)
submission_pred['label'] = [LABELS[pred] for pred in predictions]
submission_pred = pd.merge(submission_ids, submission_pred)

In [14]:
submission_path = model_path.replace(MODELS_PATH, SUBMISSIONS_PATH).replace('model_', 'submission_').replace('.h5', '.csv')
submission_pred.to_csv(submission_path, index=False)